In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.preprocessing import MinMaxScaler
from model import CalculateFeatures,SelectFeatures
%matplotlib qt


In [2]:
dtypes = {'Magnitude': np.float64, 'Latitude': np.float64, 'Longitude': np.float64, 'Depth': np.float64}
df = pd.read_csv('../../data/DATA_3_south.csv', delimiter=',', dtype=dtypes, parse_dates=["Datetime"],
                 date_parser=np.datetime64)
df = df.reset_index(drop=True)
df.head()

,Magnitude,Latitude,Longitude,Depth,Datetime
0,3.9,16.8023,-94.9130,114.9,2017-01-01 00:54:31
1,3.9,15.2842,-93.1210,100.7,2017-01-01 04:10:39
2,3.9,17.4503,-94.9543,138.3,2017-01-01 04:23:51
3,3.9,16.4077,-93.7075,139.4,2017-01-01 04:26:01
4,4.1,16.7017,-98.8633,27.5,2017-01-01 08:49:23


In [4]:
df_model = CalculateFeatures(df, 50, trim_features=True)

In [5]:
features = df_model.features.sort_values('next14dMaxMag', inplace=True)
features.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
main_features = df_model.features[
    ['meanMag', 'maxAMag', 'maxEMag', 'magDef', 'a', 'b', 'bStd', 'grcStd', 'elapsedT', 'rateSqrtEnergy', 'meanT',
     'meanTStd', 'pMag6', 'zSeismicRateChange', 'bSeismicRateChange', 'last7dMaxMag', 'next14dMaxMag']]
X = main_features.iloc[:-2]
Y = main_features.iloc[-1]
selected_features = SelectFeatures(features=X, targets=Y, corr_threshold=.95, fixed_features=["b"])[0]

X = X[[selected_features]].to_numpy()
Y = Y.to_numpy()

In [ ]:
Y_Scaler = MinMaxScaler()
X_Scaler = MinMaxScaler()
f_Y_norm = Y_Scaler.fit_transform(Y)
f_X_norm = X_Scaler.fit_transform(X)

In [ ]:
f_X_norm32 = f_X_norm.astype(np.float32)
f_Y_norm32 = f_Y_norm.astype(np.float32)

In [ ]:
from pprint import pprint

for i in range(len(X.T)):
    pprint(list(zip(f_X_norm.T[i],X.T[i],f_X_norm32.T[i])))

In [ ]:
# evaluate isomap with logistic regression algorithm for classification
pca = PCA(n_components=2)
pca.fit(f_X_norm32.T, f_Y_norm32.T)
pca_components = pca.components_

In [ ]:
cmap = cm.get_cmap("viridis")
colors = list(map(cmap, f_Y_norm32.T[0]))
sizes = np.power(10, f_Y_norm32.T[0] + .25)

In [ ]:
%matplotlib qt
plt.scatter(pca_components[0], pca_components[1], s=sizes, color=colors)

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = features.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()


In [ ]:
embedding = Isomap(n_components=2)
X_transformed = embedding.fit_transform(f_X_norm32, a)

%matplotlib qt
fix, ax = plt.subplots()
ax.set_facecolor((0, 0, .420, 0.125))
ax.scatter(X_transformed.T[0], X_transformed.T[1], s=sizes, color=colors)

In [ ]:
import matplotlib.pyplot as plt
% matplotlib qt
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(X_transformed.T[0], X_transformed.T[1], X_transformed.T[2], s=sizes, color=colors);


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

corrMatrix = main_features.corr()
sns.heatmap(corrMatrix, annot=True,cmap="bwr")
plt.show()

In [ ]:


f_X.dropna(inplace=True)
f_X.corr()

In [ ]:
sns.heatmap(pd.DataFrame(np.corrcoef(f_X.to_numpy().T)))

In [ ]:
df = f_X

feature_selection(df,.95)